# Домашнее задание 15

Добро пожаловать на пятнадцатое домашнее задание, в нем Вам предалагется найти наилучшую конфигурацию алгоритма **KNN**.

Давайте посмотрим на предложеннный набор данных!

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("15_hw_regression_dataset_prod.csv", index_col = 0)

In [3]:
df

,Col.1,Col.2,Col.3,Col.4,Col.5,Col.6,Col.7,Col.8,Col.9,Col.10,Answer
0,-1.788519,-0.894040,0.968153,-0.289677,1.380540,-1.373144,-0.320721,-0.128565,0.002373,0.703500,-3.306586
1,-0.741723,0.370529,0.694051,-0.585705,-1.070719,-0.095229,-0.358710,-0.627725,-0.208469,-0.769172,-11.020812
2,0.484705,-0.155941,-0.028969,0.279953,0.438472,0.291991,1.636664,0.869410,1.040815,0.721517,87.821276
3,0.477304,0.086616,-0.470345,0.349151,0.001512,1.653386,0.950990,-0.329011,-0.171075,-0.105319,-31.644170
4,0.049394,-0.170154,0.384732,1.039262,-1.992499,0.128568,-2.750948,-1.184568,-0.502628,-0.899456,-44.938049
...,...,...,...,...,...,...,...,...,...,...,...
995,-0.333668,-1.067437,-0.879422,-1.538592,-1.086421,-0.069894,-0.435783,-0.740377,-0.223900,-0.626643,-30.832683
996,0.323513,1.109274,0.177591,0.382659,0.746312,0.022726,1.214060,-1.611950,0.814325,0.845913,70.908163
997,0.067432,0.020427,2.052615,2.503714,1.060651,-0.589779,-0.132565,0.517800,-0.200460,-0.301766,-32.800912
998,-0.029837,-0.738788,0.266868,-0.549059,0.807669,0.423508,0.173734,0.209096,-0.702235,0.396552,-77.471404


### Задание 1

Обучите базовую конфигурацию модели с гиперпарамметрами: **n_neighbors=3, metric="euclidean"**

In [4]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor


df = pd.read_csv("15_hw_regression_dataset_prod.csv", index_col=0)

len_dataset = len(df)


predictor_cols = df.drop(["Answer"],axis=1).columns
answer_col = ["Answer"]

df_train = df.iloc[:int(len_dataset*0.9)]
df_val = df.iloc[int(len_dataset*0.9):]


X_train = df_train[predictor_cols].values
y_train = df_train[answer_col].values


X_val = df_val[predictor_cols].values
y_val = df_val[answer_col].values


neigh = KNeighborsRegressor(n_neighbors=3, metric='euclidean')
neigh.fit(X_train, y_train)

y_prediction = neigh.predict(X_val)

print("MAE =", mean_absolute_error(y_val,y_prediction))

MAE = 33.88624432934787


### Задание 2

Выполните **MinMax normalization** и обучите базовую конфигурацию модели на обновленных признаках предикторах с гиперпарамметрами: **n_neighbors=3, metric="euclidean"**

In [5]:
from sklearn.preprocessing import MinMaxScaler

def MinMax_Norm_Sklearn(df_TRAIN, df_VAL, predictors_cols_names):
    
    scaler = MinMaxScaler()

    df_TRAIN[predictors_cols_names] =  scaler.fit_transform(df_TRAIN[predictors_cols_names].values)
    df_VAL[predictors_cols_names] =  scaler.transform(df_val[predictors_cols_names].values)
    
    return df_TRAIN, df_VAL

In [6]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor


df = pd.read_csv("15_hw_regression_dataset_prod.csv", index_col=0)

len_dataset = len(df)


predictor_cols = df.drop(["Answer"],axis=1).columns
answer_col = ["Answer"]

df_train = df.iloc[:int(len_dataset*0.9)]
df_val = df.iloc[int(len_dataset*0.9):]

df_train, df_val = MinMax_Norm_Sklearn(df_train, df_val, predictor_cols)


X_train = df_train[predictor_cols].values
y_train = df_train[answer_col].values


X_val = df_val[predictor_cols].values
y_val = df_val[answer_col].values


neigh = KNeighborsRegressor(n_neighbors=3, metric='euclidean')
neigh.fit(X_train, y_train)

y_prediction = neigh.predict(X_val)

print("MAE =", mean_absolute_error(y_val,y_prediction))

MAE = 33.05300646148741


### Задание 3

Сделайте полный перебор указанных гиперпарамметров моедли и обучите для кадой конфигурации модель kNN и измерьте качество с помощью метрики MAE

In [7]:
def knn_fit_predict_mae(df_TRAIN,  df_VAL, predictors_cols_names, answer_col_name, k, metric):
    
    X_train = df_TRAIN[predictors_cols_names].values
    y_train = df_TRAIN[answer_col_name].values

    X_val = df_VAL[predictors_cols_names].values
    y_val = df_VAL[answer_col_name].values

    neigh = KNeighborsRegressor(n_neighbors=k, metric=metric)
    neigh.fit(X_train, y_train)

    prediction = neigh.predict(X_val)

    return mean_absolute_error(y_val, prediction)

In [18]:
from math import sqrt

k_list = [1, 3, 5, 9, int(sqrt(len(df_train)))]
metric_list = ["euclidean", "manhattan", "chebyshev"]
r_list = []

for k in k_list:
    for metric in metric_list:
        
        mae_k_metric = knn_fit_predict_mae(df_train,df_val,predictor_cols,answer_col,k,metric)
        
        r_list.append([mae_k_metric, k, metric])
print(sorted(r_list))

[[30.691569670033996, 9, 'manhattan'], [31.34348532162486, 5, 'manhattan'], [31.808673268589715, 5, 'euclidean'], [33.05300646148741, 3, 'euclidean'], [33.200883882150876, 3, 'manhattan'], [33.22952370987945, 9, 'euclidean'], [34.62895701725845, 30, 'manhattan'], [35.56871518768447, 30, 'euclidean'], [36.41660071399871, 9, 'chebyshev'], [38.108068960961106, 5, 'chebyshev'], [39.93914517647812, 1, 'manhattan'], [40.76463571190484, 30, 'chebyshev'], [40.84965931334018, 3, 'chebyshev'], [42.81025786205284, 1, 'euclidean'], [50.65010489124506, 1, 'chebyshev']]


### Задание 4

Выберите наилучшую конфигурацию модели kNN по метрике MAE. Заново обучите такую модель и сохраните ее ответы в новую таблицу

In [19]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor


df = pd.read_csv("15_hw_regression_dataset_prod.csv", index_col=0)

len_dataset = len(df)


predictor_cols = df.drop(["Answer"],axis=1).columns
answer_col = ["Answer"]

df_train = df.iloc[:int(len_dataset*0.9)]
df_val = df.iloc[int(len_dataset*0.9):]

df_train, df_val = MinMax_Norm_Sklearn(df_train, df_val, predictor_cols)


X_train = df_train[predictor_cols].values
y_train = df_train[answer_col].values


X_val = df_val[predictor_cols].values
y_val = df_val[answer_col].values


neigh = KNeighborsRegressor(n_neighbors=9, metric='manhattan')
neigh.fit(X_train, y_train)

y_prediction = neigh.predict(X_val)

print("MAE =", mean_absolute_error(y_val,y_prediction))

MAE = 30.691569670033996


In [20]:
df_new = df_val["Prediction"]

df_new.to_csv("h15_right_answer_prod.csv")

KeyError: 'Prediction'

И так, поздравляем, Вы выполнили пятнадцатое домашнее задание!

Теперь осталось чуть-чуть...

На платформу Школково Вам нужно будет загрузить таблицу с предсказаниями вашей лучшей модели на валидационном датасете

Чтобы сделать это, выполните ячейку выше. Тем самым Вы получите нужную таблицу. Загрузите ее на платформу Школково.